In [1]:
import pandas as pd
import autogluon
import numpy as np
import xgboost as xgb

In [2]:
test_temp_df = pd.read_parquet("./data/test_v2_1.parquet")
temp_null_idx = test_temp_df[test_temp_df["AIR_TEMPERATURE"].isna()].index
temp_null_df = test_temp_df.loc[temp_null_idx].drop(columns=["AIR_TEMPERATURE"])
temp_idx = test_temp_df[test_temp_df["AIR_TEMPERATURE"].notna()].index
test_temp_df = test_temp_df.loc[temp_idx]

In [3]:
models = []
for i in range(3):
  model = xgb.XGBRegressor()
  model.load_model(f"models/temp_model_{i}.model")
  models.append(model)
  
temps = []
for model in models:
  temps.append(model.predict(temp_null_df))

In [4]:
temp_null_df["AIR_TEMPERATURE"] = np.mean(temps, axis=0)

In [5]:
temp_result = pd.concat([test_temp_df, temp_null_df], axis=0).sort_index()

In [6]:
# U_WIND
test_uwind_df = pd.read_parquet("./data/test_v2_2.parquet").drop(columns=["V_WIND"])
uwind_null_idx = test_uwind_df[test_uwind_df["U_WIND"].isna()].index
uwind_null_df = test_uwind_df.loc[uwind_null_idx].drop(columns=["U_WIND"])
uwind_idx = test_uwind_df[test_uwind_df["U_WIND"].notna()].index
test_uwind_df = test_uwind_df.loc[uwind_idx]

models = []
for i in range(3):
  model = xgb.XGBRegressor()
  model.load_model(f"models/uwind_model_{i}.model")
  models.append(model)
  
uwinds = []
for model in models:
  uwinds.append(model.predict(uwind_null_df))
  
uwind_null_df["U_WIND"] = np.mean(uwinds, axis=0)
uwind_result = pd.concat([test_uwind_df, uwind_null_df], axis=0).sort_index()

In [7]:
# V_WIND
test_vwind_df = pd.read_parquet("./data/test_v2_2.parquet").drop(columns=["U_WIND"])
vwind_null_idx = test_vwind_df[test_vwind_df["V_WIND"].isna()].index
vwind_null_df = test_vwind_df.loc[vwind_null_idx].drop(columns=["V_WIND"])
vwind_idx = test_vwind_df[test_vwind_df["V_WIND"].notna()].index
test_vwind_df = test_vwind_df.loc[vwind_idx]

models = []
for i in range(3):
  model = xgb.XGBRegressor()
  model.load_model(f"models/uwind_model_{i}.model")
  models.append(model)
  
vwinds = []
for model in models:
  vwinds.append(model.predict(vwind_null_df))
  
vwind_null_df["U_WIND"] = np.mean(vwinds, axis=0)

vwind_result = pd.concat([test_vwind_df, vwind_null_df], axis=0).sort_index()

In [8]:
# BN
test_bn_df = pd.read_parquet("./data/test_v2_3.parquet")
bn_null_idx = test_bn_df[test_bn_df["BN"].isna()].index
bn_null_df = test_bn_df.loc[bn_null_idx].drop(columns=["BN"])
bn_idx = test_bn_df[test_bn_df["BN"].notna()].index
test_bn_df = test_bn_df.loc[bn_idx]

models = []
for i in range(3):
  model = xgb.XGBRegressor()
  model.load_model(f"models/BN_model_{i}.model")
  models.append(model)
  
BNs = []
for model in models:
  BNs.append(model.predict(bn_null_df))
  
bn_null_df["BN"] = np.mean(vwinds, axis=0)

bn_result = pd.concat([test_bn_df, bn_null_df], axis=0).sort_index()

In [12]:
temp_result["U_WIND"] = uwind_result["U_WIND"]
temp_result["V_WIND"] = vwind_result["V_WIND"]
temp_result["BN"] = bn_result["BN"]

In [14]:
temp_result.columns

Index(['AIR_TEMPERATURE', 'ARI_CO_BR', 'ARI_CO_CA', 'ARI_CO_CL', 'ARI_CO_CN',
       'ARI_CO_FI', 'ARI_CO_ID', 'ARI_CO_IN', 'ARI_CO_JP', 'ARI_CO_KR',
       ...
       'WTI', 'day', 'hour', 'minute', 'month', 'weekday', 'year', 'U_WIND',
       'V_WIND', 'BN'],
      dtype='object', length=283)

In [15]:
temp_result.to_parquet("data/test_v2.parquet")